#### The code sets up a query engine (query_engine) using a VectorStoreIndex and a language model.

The primary purpose of this code is to implement a Retrieval-Augmented Generation (RAG) system capable of answering questions based on the content of personal PDF documents. These documents are processed and converted into embeddings, indexed using LlamaIndex, and stored in a persistent vector database. The vector database (ChromaDB) was pre-populated in an earlier step, likely using the ‘run_first_prepare_data.ipynb’ notebook.

This implementation leverages the OpenAI embedding model (OpenAIEmbedding) to encode the content into vector representations and the Groq language model to generate human-like responses. The vector database serves as the backbone of the RAG system, enabling efficient similarity searches and retrieval of relevant information to enhance the accuracy and context of the generated answers.

In [ ]:
import os
from warnings import filterwarnings
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.groq import Groq
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from icecream import ic

In [2]:
INPUT_PDF_DIR = 'input/pdfs/'
OUTPUT_MD_DIR = 'input/mds/'
CHROMADB_DIR = 'database/vector_store/'
CHROMADB_COLLECTION = 'rag_collection'

load_dotenv()

filterwarnings(action="ignore", category=FutureWarning, module="easyocr")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

#### The code then defines a query engine (query_engine) using the VectorStoreIndex and the language model.

The purpose of this code is likely to set up a RAG system that can answer questions based on the content of personal PDF documents, which are processed and indexed using LlamaIndex, leveraging the vector database created in the previous step by the 'run_first_prepare_data.ipynb' notebook."

This adds the information that the vector database created in the previous step is being used to power the RAG system.

In [3]:
chroma_embed_model = OpenAIEmbedding(api_key=os.getenv("OPENAI_API_KEY"))
llm_model = Groq(model="llama3-70b-8192", api_key=os.getenv("GROQ_API_KEY"))

chroma_client = chromadb.PersistentClient(path = CHROMADB_DIR)
chroma_collection = chroma_client.get_or_create_collection(name=CHROMADB_COLLECTION)

vector_store = ChromaVectorStore(chroma_collection = chroma_collection)
storage_context = StorageContext.from_defaults(vector_store = vector_store)
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context, embed_model=chroma_embed_model)

query_engine = index.as_query_engine(llm = llm_model)

In [ ]:
QUERY = 'YOUR FIRST QUESTION HERE?'
result = query_engine.query(QUERY)
ic(result.response)